In [1]:
import matplotlib.pyplot as plt
#%matplotlib inline

from skimage import data
from skimage import color
from skimage import morphology
from skimage import segmentation
from skimage import filters,io
from skimage.future import graph
from skimage.measure import regionprops, regionprops_table
from skimage import draw
from skimage import feature
import networkx as nx

from scipy import ndimage as ndi
from scipy.spatial.distance import directed_hausdorff

from skimage.util import img_as_float
from skimage.filters import gabor_kernel

import pickle

import math


Bad key "text.kerning_factor" on line 4 in
/home/ubuntu/anaconda3/envs/graph/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import tensorflow as tf
from keras import Input, Model
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2

from spektral.layers import GraphConv, TopKPool,DiffPool, SAGPool,MinCutPool
from spektral.layers import GraphAttention, GlobalAttentionPool
from spektral.layers.ops import sp_matrix_to_sp_tensor
from spektral.utils import normalized_laplacian


Using TensorFlow backend.


In [3]:

HOME_DIR = "/home/ubuntu/"


INPUT = HOME_DIR + "Dataset/fields_with_masks/"

Defined_CLASSES = [1]
BATCH_SIZE = 2
EPOCHS=10

In [4]:
# Parameters
l2_reg = 5e-4         # Regularization rate for l2
learning_rate = 1e-3  # Learning rate for SGD
batch_size = 2       # Batch size
epochs = 10       # Number of training epochs
es_patience = 200     # Patience fot early stopping

In [5]:
import matplotlib.pyplot as plt
from io import BytesIO
import matplotlib.image as mpimg
import numpy as np
import cv2
cv2.ocl.setUseOpenCL(True)

import os

import tensorflow as tf
import keras
from keras.backend.tensorflow_backend import set_session  

config =  tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)
set_session(sess)

""" This function used to visualize the input, predicted mask and ground truth"""
def visualize(save_status, save_path,flight_code,**images ):
    
    n = len(images)
    plt.figure(figsize=(40,40))
    for i, (name , image) in enumerate(images.items()):
        plt.subplot(1, n, i+1)
        plt.xticks([])
        plt.yticks([])
        # plt.title(str(i+1))
        plt.title(' '.join(name.split('_')).title())
        
        plt.imshow(image,cmap='gray')
    
    if save_status : 
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        plt.savefig(save_path+flight_code+'.jpg')    
    plt.show()
    
    
def denormalize(x):
    x_max = np.max(x)
    x_min = np.min(x)
    x = (x-x_min) / (x_max - x_min)
    x = x.clip(0,1)
    return x

In [6]:
from statistics import mean 
def compute_feats(image, kernels):
    feats = []
    for k, kernel in enumerate(kernels):
        filtered = ndi.convolve(image, kernel, mode='wrap')
        feats.append(filtered.mean())
        feats.append(filtered.std())
    return feats
def compute_rgb_hist_feats(image):
    chans = cv2.split(image)

    features = []
    for chan in chans:
        hist = cv2.calcHist([chan], [0], None, [256], [0, 256])
        cv2.normalize(hist,hist, 0, 255, cv2.NORM_MINMAX)
        j = []
        for i in hist:
            j.append(np.mean(i))
        j = np.asarray(j)    
        features.append(np.mean(j))
        
    return features
def make_ganbor_bank_kernels():
    kernels = []
    for theta in range(4):
        theta = theta / 4. * np.pi
        for sigma in (1, 3):
            for frequency in (0.05, 0.25):
                kernel = np.real(gabor_kernel(frequency, theta=theta,
                                              sigma_x=sigma, sigma_y=sigma))
                kernel = np.stack((kernel,kernel,kernel),axis = 2)
                #print(kernel.shape)
                kernels.append(kernel)
    return kernels

In [7]:
kernels = make_ganbor_bank_kernels()
def find_bounding_contour_and_cal_features(segments,image, prev_node_no,nutrient_mask):
    # loop over the unique segment values
    image_copy = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    all_features = []
    all_labels= []
    ##for node 0
    list_zeros = (np.zeros(9)).tolist()
    all_features.append(list_zeros)
    all_labels.append(0)
    for (i, segVal) in enumerate(np.unique(segments)):
    # construct a mask for the segment
        if i > 0:
            #print("[x] inspecting segment %d" % (i))
            mask = np.zeros(image_copy.shape[:2], dtype = "uint8")
            mask[segments == segVal] = 255
        #   # show the masked region
        #   cv2.imshow("Mask", mask)
        #   cv2.imshow("Applied", cv2.bitwise_and(image, image, mask = mask))
            result = cv2.bitwise_and(image_copy, image_copy, mask = mask)
            #print(cv2.findContours(result,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE))
            cnt = cv2.findContours(result,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            x,y,w,h = cv2.boundingRect(cnt[0])
            #print('x:{},y:{},w:{},h:{}'.format(x,y,w,h))
            roi = image[y:y+w, x:x+h]
            roi_mask = nutrient_mask[y:y+w, x:x+h]
            roi_feats =[]#compute_feats(image=roi, kernels=kernels)
            roi_feats.extend((np.mean(roi, axis=(0, 1))).tolist())
            roi_feats.extend((np.std(roi, axis=(0, 1))).tolist())
            roi_feats.extend((compute_rgb_hist_feats(roi)))
            all_features.append(roi_feats)
            if cv2.countNonZero(roi_mask) > 0: #0.5*w*h:
                label = cv2.countNonZero(roi_mask) / (w*h)
                all_labels.append(label)
            else:
                all_labels.append(0)
            #if image is not None:
                #image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
                #cv2.drawContours(image, cnt[0], -1, (0,255,0), 1)
                #cv2.rectangle(image,(x,y),(x+w,y+h), (255,0,0), 2)
                #cv2.imshow("", image)
               # cv2.imshow("roi",roi)
                #cv2.waitKey(1000)
        #   cv2.waitKey(0)
    while prev_node_no < 401:
        list_zero = (np.zeros(9)).tolist()
        all_features.append(list_zeros)
        all_labels.append(0)
        prev_node_no += 1
    return all_features, all_labels

In [8]:
def display_edges(image, g, threshold):
    edges = []
    for n1, n1_neigh in g._adj.items():
        for n2 in n1_neigh:
            #print(str(n1) + ':', str(n2))
            edges.append((n1,n2))

    image = image.copy()
    for edge in edges:
        n1, n2 = edge
 
        r1, c1 = map(int, g._node[n1]['centroid'])
        r2, c2 = map(int, g._node[n2]['centroid'])
 
        line  = draw.line(r1, c1, r2, c2)
        circle = draw.circle(r1,c1,4)
 
        if g._adj[n1][n2]['weight'] < threshold :
            image[line] = 0,1,1
        image[circle] = 1,1,0
 
    return image

def rag_mean_distance_color(image, labels, connectivity=1, mode='distance', sigma=255.0, threshold=5):
    def np_hist_to_cv(np_histogram_output):
        counts, bin_edges = np_histogram_output
        return counts.ravel().astype('float32')
    
    image_copy = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    g = graph.RAG(labels, connectivity=connectivity)

    for n in g:
        g.nodes[n].update({'labels': [n],
                               'pixel count': 0,
                               'total color': [],
                               'hist': None})#np.array([0, 0, 0],
                                                      #dtype=np.double)})

    for index in np.ndindex(labels.shape):
        current = labels[index]
        g.nodes[current]['pixel count'] += 1
        g.nodes[current]['total color'].append(image[index])
    for n in g:
        roi = np.asarray(g.nodes[n]['total color'], dtype=np.float32)
        g.nodes[n]['hist'] = np.histogramdd(roi)
        #print(np_hist_to_cv(g.nodes[n]['hist']))
#         g.nodes[n]['mean color'] = (g.nodes[n]['total color'] /
#                                        g.nodes[n]['pixel count'])
    #all_sim = []
    for x in g:
        for y in g:
            if x != y:
                diff = cv2.compareHist(np_hist_to_cv(g.nodes[x]['hist']), np_hist_to_cv(g.nodes[y]['hist']), cv2.HISTCMP_BHATTACHARYYA)
                #diff = np.linalg.norm(diff)
                if mode == 'similarity':
                    diff = math.e ** (-(diff ** 2) / sigma)
                elif mode =='distance':
                    diff = diff
                else:
                    raise ValueError("The mode '%s' is not recognised" % mode)
                sim = 1-diff
                #all_sim.append(sim)
                if sim >= threshold:
                    if g.has_edge(x,y):
                        g.edges[x, y]['weight'] = sim
                    else:
                        g.add_edge(x, y, weight=sim )
                else:
                    if g.has_edge(x,y):
                        g.remove_edge(x, y)
    return g#,all_sim

def super_pixels(**kwargs):
    
    def show_img(img):
        width = 10.0
        height = img.shape[0]*width/img.shape[1]
        f = plt.figure(figsize=(width, height))
        plt.imshow(img)
    
    image = kwargs.get('image', None)
    #show_img(image)
    mask = kwargs.get('mask',None)
    
    nutrient_mask = kwargs.get('nutrient_mask', None)
    labels = segmentation.slic(image, mask = mask, compactness=30, n_segments=400)
    #show_img(labels)
    regions = regionprops(labels)
    #regions_table = regionprops_table(labels)
    #print(regions_table)
    label_rgb = color.label2rgb(labels, image, kind='avg',bg_label=0)
    
    #show_img(label_rgb)
    
    label_rgb = segmentation.mark_boundaries(label_rgb, labels, (0, 0, 0))
    
    #rag = graph.rag_mean_color(image, labels,connectivity = 1 , mode = 'distance')#'similarity', sigma =255)
    rag= rag_mean_distance_color(image, labels,connectivity = 1 , mode = 'distance', threshold=0 )
    #plt.figure()
    #plt.hist(all_diff, 100)
    #plt.show()
    prev_node_no = len(rag)
    rag.remove_node(0)
    rag.add_node(0)
    while len(rag) < 401:
        rag.add_node(len(rag),size=0)   
#     for region in regions:
#         #print(region['centroid'])
#         rag._node[region['label']]['centroid'] = region['centroid']
    A =nx.adjacency_matrix(rag)
    A = A.toarray()
    #A = normalized_laplacian(A)
    X,y=find_bounding_contour_and_cal_features(labels,image,prev_node_no,nutrient_mask)
    X= np.array(X)
    y = np.array(y)
    
    #edges_drawn_all = display_edges(label_rgb, rag, np.inf )
    #show_img(edges_drawn_all)
    
    return A,labels,X,y

In [9]:
import re   
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key,reverse=True)     
class Dataset:
    
    CLASSES = Defined_CLASSES
    def __init__(
            self,
            field_dir,
            status, #train,test,validation
            classes = None,
            augmentation = None,
            preprocessing = None
            ):
        """ make a list of image directions  """
        files = ['bounday_mask.png','nutrient_mask.png']#,'ndvi.png','gndvi.png','ndwi.png']
        self.field_dirs = [ids for ids in sorted(os.listdir(field_dir))]
        self.field_fps = [os.path.join(field_dir ,field_id) for field_id in self.field_dirs if len(os.listdir(os.path.join(field_dir ,field_id))) >= 3 ]
        
        train, validate, test = np.split(self.field_fps, [int(.6*len(self.field_fps)), int(.8*len(self.field_fps))])
        print(len(train))
        print(len(validate))
        print(len(test))
        if status == 'train':
            self.field_fps = train
        elif status == 'validate':
            self.field_fps = validate
        elif status == 'test':
            self.field_fps = test
        
        self.field_codes            = []
        self.field_number            = []
        self.field_images            = []
        self.field_nutrient_mask     = []
        self.field_boundary_mask     = []

        for field_path in self.field_fps:
            #if(field_path != '/home/ubuntu/Dataset/fields_with_masks/8849'):
                #print('yes')
            flight_dirs = sorted_alphanumeric(os.listdir(field_path))
            codes  = []
            number = []
            images = []
            nutrient_mask = []
            boundary_mask = []

            for j in flight_dirs:
                flight_number, flight_code = j.split('_')
                codes.append(flight_code)
                number.append(flight_number)
                images.append(os.path.join(field_path,j,flight_code+'.png'))
                boundary_mask.append(os.path.join(field_path,j,files[0]))
                nutrient_mask.append(os.path.join(field_path,j,files[1]))


            self.field_codes.append(codes)
            self.field_number.append(number)
            self.field_images.append(images)
            self.field_nutrient_mask.append(nutrient_mask)
            self.field_boundary_mask.append(boundary_mask)

        
        """ set the class values and assign a augmentation and preprocessing method"""
        # self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        self.class_values = classes
        self.augmentation = augmentation
        self.preprocessing = preprocessing
        
    def __getitem__(self,i):
         
        """ read images"""
        image1 = cv2.imread(self.field_images[i][0])
        image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    
    
        
        boundary_mask = cv2.imread(self.field_boundary_mask[i][0])
        boundary_mask = cv2.cvtColor(boundary_mask, cv2.COLOR_BGR2GRAY)
        
        
        nutrient_mask = cv2.imread(self.field_nutrient_mask[i][0])
        nutrient_mask = cv2.cvtColor(nutrient_mask, cv2.COLOR_BGR2GRAY) 
        
        boundary_mask = denormalize(boundary_mask)
        nutrient_mask = denormalize(nutrient_mask)

        
        """ extract certain classes from mask 1 """
        masks = [(boundary_mask == v) for v in self.class_values]
        boundary_mask = np.stack(masks, axis = -1).astype('float')
        
        boundary_mask_for_slic = np.stack(masks, axis = 0).astype('float')
        
        masks = [(nutrient_mask == v) for v in self.class_values]
        nutrient_mask = np.stack(masks, axis = -1).astype('float')
        
        data1 = {"image": image1, "mask": nutrient_mask }
        """ apply augmentation """
        if self.augmentation:
            sample = self.augmentation(**data1)
            image1 , nutrient_mask = sample['image'], sample['mask']
         
        data2 = {"image": image1, "mask": nutrient_mask }
        """ apply preprocessing """
        if self.preprocessing:
            sample = self.preprocessing(**data2)
            image1 , nutrient_mask = sample['image'], sample['mask']
        
        A, labels,X,y = super_pixels(image=image1, mask=boundary_mask_for_slic, nutrient_mask=nutrient_mask)
             
        #return image, nutrient_mask, self.image_dirs[i]
        return A, labels, X,self.field_codes[i][0] ,y,nutrient_mask
    
    def __len__(self):
        return len(self.field_images)
    


In [10]:
import boto3, os
import matplotlib.pyplot as plt
import pickle

# s3client = boto3.client('s3')

# response = s3client.get_object(Bucket='flight-ops', Key='saba.dadsetan/third_chunck_train_5.pickle')

# body = response['Body'].read()
# data = pickle.loads(body)


# flight_codes = []
# Adjacency = []
# features = []
# pixel_labels=[]
# y_label = []
# nutrient_mask = []
# for code in data.keys():
#     flight_codes.append(code)
#     Adjacency.append(data[code]['A'])
#     features.append(data[code]['X'])
#     pixel_labels.append(data[code]['pixel_labels'])
#     y_label.append(data[code]['Y'])
#     nutrient_mask.append(data[code]['nutrient_mask'])
    
    
bucket='flight-ops'


In [11]:
import pickle
dataset = Dataset(INPUT,status='train', classes = Defined_CLASSES)
# flights = {}
# key='saba.dadsetan/pickles/first_chunk_train.pickle'
# for i in range(0,75):
#     print(i)
#     A,pixel_labels, X, flight_code,Y,nutrient_mask = dataset[i]
#     a = {'A':A, 'pixel_labels':pixel_labels, 'X':X, 'Y':Y, 'nutrient_mask':nutrient_mask}
#     flights[flight_code]= a

# pickle_byte_obj = pickle.dumps(flights, protocol=pickle.HIGHEST_PROTOCOL)

# s3_resource = boto3.resource('s3')
# s3_resource.Object(bucket,key).put(Body=pickle_byte_obj)
# print('First Chunck Done... :D ')
# del(flights)

# flights = {}
# key='saba.dadsetan/pickles/second_chunk_train.pickle'
# for i in range(75,150):
#     print(i)
#     A,pixel_labels, X, flight_code,Y,nutrient_mask = dataset[i]
#     a = {'A':A, 'pixel_labels':pixel_labels, 'X':X, 'Y':Y, 'nutrient_mask':nutrient_mask}
#     flights[flight_code]= a
# pickle_byte_obj = pickle.dumps(flights, protocol=pickle.HIGHEST_PROTOCOL)

# s3_resource = boto3.resource('s3')
# s3_resource.Object(bucket,key).put(Body=pickle_byte_obj)
# print('Second Chunck Done... :D ')
# del(flights)

flights = {}
key='saba.dadsetan/pickles/third_chunk_train.pickle'
for i in range(150,len(dataset)):
    print(i)
    A,pixel_labels, X, flight_code,Y,nutrient_mask = dataset[i]
    a = {'A':A, 'pixel_labels':pixel_labels, 'X':X, 'Y':Y, 'nutrient_mask':nutrient_mask}
    flights[flight_code]= a
pickle_byte_obj = pickle.dumps(flights, protocol=pickle.HIGHEST_PROTOCOL)

s3_resource = boto3.resource('s3')
s3_resource.Object(bucket,key).put(Body=pickle_byte_obj)
print('Third Chunck Done... :D ')
del(flights)

231
77
78
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194


/home/ubuntu/anaconda3/envs/graph/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ubuntu/anaconda3/envs/graph/lib/python3.6/site-packages/numpy/core/_methods.py:154: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/home/ubuntu/anaconda3/envs/graph/lib/python3.6/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/ubuntu/anaconda3/envs/graph/lib/python3.6/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/ubuntu/anaconda3/envs/graph/lib/python3.6/site-packages/numpy/core/_methods.py:207: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
Third Chunck Done... :D 


In [12]:
import pickle
dataset = Dataset(INPUT,status='test', classes = Defined_CLASSES)
flights = {}
key='saba.dadsetan/pickles/test.pickle'
for i in range(0,len(dataset)):
    print(i)
    A,pixel_labels, X, flight_code,Y,nutrient_mask = dataset[i]
    a = {'A':A, 'pixel_labels':pixel_labels, 'X':X, 'Y':Y, 'nutrient_mask':nutrient_mask}
    flights[flight_code]= a
pickle_byte_obj = pickle.dumps(flights,protocol=pickle.HIGHEST_PROTOCOL)

s3_resource = boto3.resource('s3')
s3_resource.Object(bucket,key).put(Body=pickle_byte_obj)
print('test Chunck Done... :D ')
del(flights)


231
77
78
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
test Chunck Done... :D 


In [13]:
import pickle
Dataset(INPUT,status='validate', classes = Defined_CLASSES)
flights = {}
key='saba.dadsetan/pickles/validate.pickle'
for i in range(0,len(dataset)):
    print(i)
    A,pixel_labels, X, flight_code,Y,nutrient_mask = dataset[i]
    a = {'A':A, 'pixel_labels':pixel_labels, 'X':X, 'Y':Y, 'nutrient_mask':nutrient_mask}
    flights[flight_code]= a
pickle_byte_obj = pickle.dumps(flights, protocol=pickle.HIGHEST_PROTOCOL)

s3_resource = boto3.resource('s3')
s3_resource.Object(bucket,key).put(Body=pickle_byte_obj)
print('validate Chunck Done... :D ')
del(flights)


231
77
78
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
validate Chunck Done... :D 
